This notebook demonstrates the Finite-Interval Forecasting Engine (FIFE).

FIFE is a Python package developed by the Institute for Defense Analyses in a research partnership with the U.S. Office of the Undersecretary of Defense for Personnel and Readiness.

FIFE applies state-of-the-art forecasting methods to [panel data](https://en.wikipedia.org/wiki/Panel_data). A panel dataset contains periodic observations of subjects. In a given period, subjects may enter or exit the panel; subjects that do not exit survive to be observed in later periods. FIFE can forecast subject-specific probabilities of survival, future states of survival, and circumstances of exit for each future period up to a given or inferred maximum time horizon.

In this example we forecast when each leader of the world's countries will lose power. To skip past how FIFE works to what FIFE can produce, see the [Forecasts section](#Forecasts).

There are many aspects of FIFE that this example does not cover; you can read about those aspects in [FIFE's documentation](https://fife.readthedocs.io/en/latest/).

We start by importing the objects we'll be using.

In [1]:
from fife.lgb_modelers import LGBSurvivalModeler
from fife.processors import PanelDataProcessor
from fife.utils import make_results_reproducible
from pandas import concat, date_range, read_csv, to_datetime

The following makes sure we get the same results each time we execute this code from top to bottom.

In [2]:
SEED = 9999
make_results_reproducible(SEED)

We use the July 2020 edition of the [Rulers, Elections, and Irregular Governance dataset (REIGN)](https://oefdatascience.github.io/REIGN.github.io/), a monthly panel of national leaders and political conditions since January 1950. We load the REIGN data directly from its online archive. If you plan to execute this notebook repeatedly, please avoid spamming the REIGN archive with requests by saving the data locally, then changing the file path in the cell below to your local path.

In [3]:
data = read_csv("https://www.dl.dropboxusercontent.com/s/3tdswu2jfgwp4xw/REIGN_2020_7.csv?dl=0")
data.head()

,ccode,country,leader,year,month,elected,age,male,militarycareer,tenure_months,...,delayed,lastelection,loss,irregular,prev_conflict,pt_suc,pt_attempt,precip,couprisk,pctile_risk
0,2.0,USA,Truman,1950.0,1.0,1.0,66.0,1,0.0,58.0,...,0.0,2.639057,5.327876,7.565793,0.0,0.0,0.0,-0.069058,NaN,NaN
1,2.0,USA,Truman,1950.0,2.0,1.0,66.0,1,0.0,59.0,...,0.0,2.708050,5.332719,7.566311,0.0,0.0,0.0,-0.113721,NaN,NaN
2,2.0,USA,Truman,1950.0,3.0,1.0,66.0,1,0.0,60.0,...,0.0,2.772589,5.337538,7.566829,0.0,0.0,0.0,-0.108042,NaN,NaN
3,2.0,USA,Truman,1950.0,4.0,1.0,66.0,1,0.0,61.0,...,0.0,2.833213,5.342334,7.567346,0.0,0.0,0.0,-0.041600,NaN,NaN
4,2.0,USA,Truman,1950.0,5.0,1.0,66.0,1,0.0,62.0,...,0.0,2.890372,5.347107,7.567863,0.0,0.0,0.0,-0.129703,NaN,NaN


The data need only minimal pre-processing to be ready for FIFE. Panel data are defined by two identifiers: individual and time. In the REIGN data, we can identify an individual by their name and the country they lead. We can identify time by year and month. FIFE will automatically infer the individual and time identifiers as the two left-most columns in the data, so we rearrange the columns accordingly. Along the way we drop the redundant columns `ccode` (a numeric code for country) and `leader`.

In [4]:
data["country-leader"] = data["country"] + ": " + data["leader"]
data["year-month"] = data["year"].astype(int).astype(str) + data["month"].astype(int).astype(str).str.zfill(2)
data["year-month"] = to_datetime(data["year-month"], format="%Y%m")
data = concat([data[["country-leader", "year-month"]],
               data.drop(["ccode", "country-leader", "leader", "year-month"],
                         axis=1)],
               axis=1)
data.head()

,country-leader,year-month,country,year,month,elected,age,male,militarycareer,tenure_months,...,delayed,lastelection,loss,irregular,prev_conflict,pt_suc,pt_attempt,precip,couprisk,pctile_risk
0,USA: Truman,1950-01-01,USA,1950.0,1.0,1.0,66.0,1,0.0,58.0,...,0.0,2.639057,5.327876,7.565793,0.0,0.0,0.0,-0.069058,NaN,NaN
1,USA: Truman,1950-02-01,USA,1950.0,2.0,1.0,66.0,1,0.0,59.0,...,0.0,2.708050,5.332719,7.566311,0.0,0.0,0.0,-0.113721,NaN,NaN
2,USA: Truman,1950-03-01,USA,1950.0,3.0,1.0,66.0,1,0.0,60.0,...,0.0,2.772589,5.337538,7.566829,0.0,0.0,0.0,-0.108042,NaN,NaN
3,USA: Truman,1950-04-01,USA,1950.0,4.0,1.0,66.0,1,0.0,61.0,...,0.0,2.833213,5.342334,7.567346,0.0,0.0,0.0,-0.041600,NaN,NaN
4,USA: Truman,1950-05-01,USA,1950.0,5.0,1.0,66.0,1,0.0,62.0,...,0.0,2.890372,5.347107,7.567863,0.0,0.0,0.0,-0.129703,NaN,NaN


By definition, each unique pair of identifier values identifies a single observation. REIGN contains a few violations of this definition, due to leaders losing and regaining power in a given month. We keep only the first observation of a given leader in a given month, thereby ignoring intra-month lapses in power.

In [5]:
total_obs = len(data)
data = data.drop_duplicates(["country-leader", "year-month"], keep="first")
n_duplicates = total_obs - len(data)
print(f"{n_duplicates} observations with a duplicated identifier pair deleted.")

7 observations with a duplicated identifier pair deleted.


FIFE's PanelDataProcessor does the rest of our data processing for us, including identifying which features are categorical, computing which leadership spells ended and when, and reserving a random 25% of leaders for model validation.

In [6]:
data_processor = PanelDataProcessor(data=data)
data_processor.build_processed_data()
data_processor.data.head()

Time identifier column name not given; assumed to be second-leftmost column (year-month)
Individual identifier column name not given; assumed to be leftmost column (country-leader)


,country-leader,year-month,country,year,month,elected,age,male,militarycareer,tenure_months,...,precip,couprisk,pctile_risk,_period,_predict_obs,_test,_validation,_maximum_lead,_duration,_event_observed
0,Afghanistan: Abdallah Yakta,1967-10-01,Afghanistan,1967,10,0,53,1,0,1,...,0.018704,NaN,NaN,213,False,False,False,633,1,True
1,Afghanistan: Abdallah Yakta,1967-11-01,Afghanistan,1967,11,0,53,1,0,2,...,0.179240,NaN,NaN,214,False,False,False,632,0,True
2,Afghanistan: Abdul Zahir,1971-06-01,Afghanistan,1971,6,0,61,1,0,1,...,-1.806554,NaN,NaN,257,False,False,False,589,18,True
3,Afghanistan: Abdul Zahir,1971-07-01,Afghanistan,1971,7,0,61,1,0,2,...,-1.797812,NaN,NaN,258,False,False,False,588,17,True
4,Afghanistan: Abdul Zahir,1971-08-01,Afghanistan,1971,8,0,61,1,0,3,...,-1.815134,NaN,NaN,259,False,False,False,587,16,True


We pass the processed data to FIFE's LGBSurvivalModeler, which trains gradient-boosted tree models using [LightGBM](https://lightgbm.readthedocs.io/en/latest/). FIFE trains one model for each time horizon up to some maximum. In the absence of a specified maximum, FIFE will train up to the time horizon for which there are no more than 64 training observations that survived that long.

By default, FIFE uses [Dask](https://dask.org/) to parallelize training across time horizons, which can speed up the modeling process. However, parallelization can cause an [IOStream issue endemic to Jupyter notebooks](https://github.com/ipython/ipykernel/issues/334). We can tell FIFE to train one model at a time by specifying `parallelize=False`. As a side effect, we can view the log of the modeling process in logical order. Each line reports the model loss on the validation set after training a new tree. Training stops once the validation loss hasn't improved for four trees in a row. Then FIFE starts training trees for the next time horizon, until it reaches the maximum.

In [7]:
modeler = LGBSurvivalModeler(data=data_processor.data)
modeler.build_model(parallelize=False)

Time identifier column name not given; assumed to be second-leftmost column (year-month)
Individual identifier column name not given; assumed to be leftmost column (country-leader)


Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']


[1]	validation_set's binary_logloss: 0.0618062
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0584955
[3]	validation_set's binary_logloss: 0.0572142
[4]	validation_set's binary_logloss: 0.0562582
[5]	validation_set's binary_logloss: 0.0554188
[6]	validation_set's binary_logloss: 0.0548194
[7]	validation_set's binary_logloss: 0.0542496
[8]	validation_set's binary_logloss: 0.0536418
[9]	validation_set's binary_logloss: 0.0532795
[10]	validation_set's binary_logloss: 0.0529108
[11]	validation_set's binary_logloss: 0.0526117
[12]	validation_set's binary_logloss: 0.0524097
[13]	validation_set's binary_logloss: 0.0520414
[14]	validation_set's binary_logloss: 0.0517947
[15]	validation_set's binary_logloss: 0.0515386
[16]	validation_set's binary_logloss: 0.0513138
[17]	validation_set's binary_logloss: 0.0512088
[18]	validation_set's binary_logloss: 0.0510135
[19]	validation_set's binary_logloss: 0.0507997
[20]	validation_set's binary_logloss:

Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']


[1]	validation_set's binary_logloss: 0.071601
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0694185
[3]	validation_set's binary_logloss: 0.0679357
[4]	validation_set's binary_logloss: 0.0669349
[5]	validation_set's binary_logloss: 0.0661148
[6]	validation_set's binary_logloss: 0.0654482
[7]	validation_set's binary_logloss: 0.0648559
[8]	validation_set's binary_logloss: 0.0642733
[9]	validation_set's binary_logloss: 0.063824
[10]	validation_set's binary_logloss: 0.0634412
[11]	validation_set's binary_logloss: 0.0630345
[12]	validation_set's binary_logloss: 0.0627608
[13]	validation_set's binary_logloss: 0.0625147
[14]	validation_set's binary_logloss: 0.0623169
[15]	validation_set's binary_logloss: 0.0621811
[16]	validation_set's binary_logloss: 0.0620206
[17]	validation_set's binary_logloss: 0.0618141
[18]	validation_set's binary_logloss: 0.0616092
[19]	validation_set's binary_logloss: 0.0615297
[20]	validation_set's binary_logloss: 0

...

[1]	validation_set's binary_logloss: 0.047578
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.047594
[3]	validation_set's binary_logloss: 0.0473391
[4]	validation_set's binary_logloss: 0.0482845
[5]	validation_set's binary_logloss: 0.0484176
[6]	validation_set's binary_logloss: 0.0481604
[7]	validation_set's binary_logloss: 0.0477097
Early stopping, best iteration is:
[3]	validation_set's binary_logloss: 0.0473391


Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']


[1]	validation_set's binary_logloss: 0.0416395
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0428448
[3]	validation_set's binary_logloss: 0.0444097
[4]	validation_set's binary_logloss: 0.045517
[5]	validation_set's binary_logloss: 0.0454223
Early stopping, best iteration is:
[1]	validation_set's binary_logloss: 0.0416395


Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']


[1]	validation_set's binary_logloss: 0.0434231
Training until validation scores don't improve for 4 rounds
[2]	validation_set's binary_logloss: 0.0428569
[3]	validation_set's binary_logloss: 0.0435732
[4]	validation_set's binary_logloss: 0.044069
[5]	validation_set's binary_logloss: 0.0437817
[6]	validation_set's binary_logloss: 0.0435696
Early stopping, best iteration is:
[2]	validation_set's binary_logloss: 0.0428569


#### Forecasts
Obtaining a forecast for every current leader is as simple as `modeler.forecast()`. We can also customize the column headers of the forecasts. Each value represents the probability that the given leader will still be in power at the start of the given month.

In [8]:
forecasts = modeler.forecast()
custom_forecast_headers = date_range(data["year-month"].max(),
                                     periods=len(forecasts.columns) + 1,
                                     freq="M").strftime("%b %Y")[1:]
forecasts.columns = custom_forecast_headers
forecasts

,Aug 2020,Sep 2020,Oct 2020,Nov 2020,Dec 2020,Jan 2021,Feb 2021,Mar 2021,Apr 2021,May 2021,...,Dec 2039,Jan 2040,Feb 2040,Mar 2040,Apr 2040,May 2040,Jun 2040,Jul 2040,Aug 2040,Sep 2040
country-leader,,,,,,,,,,,,,,,,,,,,,
Afghanistan: Ashraf Ghani,0.993990,0.970821,0.946974,0.925935,0.912296,0.903354,0.898115,0.883586,0.876560,0.867222,...,0.016121,0.015504,0.015401,0.015211,0.013763,0.013671,0.013586,0.013509,0.013416,0.013333
Albania: Rama,0.994358,0.989848,0.986320,0.982195,0.974887,0.968206,0.960713,0.952167,0.941558,0.920376,...,0.005599,0.005563,0.005503,0.005465,0.005341,0.005305,0.005272,0.005242,0.005206,0.005167
Algeria: Tebboune,0.993810,0.979048,0.973364,0.958141,0.947408,0.933675,0.924000,0.914612,0.908370,0.898020,...,0.105963,0.105271,0.104574,0.103860,0.103159,0.102470,0.101829,0.101255,0.100558,0.099937
Andorra: Espot Zamora,0.985801,0.978421,0.970158,0.956953,0.946031,0.931912,0.926487,0.919690,0.910704,0.902226,...,0.023392,0.023238,0.023084,0.022927,0.022772,0.022620,0.022479,0.022352,0.022198,0.022024
Angola: Lourenco,0.975911,0.921862,0.909518,0.905946,0.897450,0.884904,0.878615,0.869160,0.861654,0.853856,...,0.059059,0.058673,0.058286,0.057889,0.055939,0.055566,0.055218,0.054907,0.054529,0.054186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela: Nicolas Maduro,0.997543,0.993290,0.990913,0.988686,0.984530,0.981881,0.978523,0.973407,0.939389,0.932764,...,0.083788,0.083239,0.082688,0.080565,0.078729,0.078215,0.077726,0.077287,0.076755,0.076282
Vietnam: Phu Trong,0.983186,0.976467,0.971653,0.818323,0.811781,0.795067,0.789577,0.780696,0.724950,0.720005,...,0.045882,0.045570,0.045259,0.044951,0.044647,0.044349,0.042229,0.041921,0.041632,0.041371
Yemen: Houthi,0.998209,0.983337,0.974691,0.966096,0.958313,0.946798,0.937605,0.927932,0.911492,0.901282,...,0.117645,0.116873,0.116099,0.115307,0.114528,0.113764,0.110606,0.109982,0.109242,0.108583


We won't know how accurate these forecasts are until the future reveals itself to us. However, we can *pretend* to not have the most recent, say, 36 months of data, train a model, then evaluate that model's forecasts over those 36 months. Pretending to not have the most recent 36 months of data is as easy as specifying the TEST_INTERVALS configuration parameter and re-computing leadership spells.

In [9]:
TEST_INTERVALS = 36
data_processor.config["TEST_INTERVALS"] = TEST_INTERVALS
data_processor.build_reserved_cols()

modeler = LGBSurvivalModeler(data=data_processor.data)
modeler.build_model(n_intervals=TEST_INTERVALS)

Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']


[1]	validation_set's binary_logloss: 0.0641882
Training until validation scores don't improve for 4 rounds
[1]	validation_set's binary_logloss: 0.0636675[1]	validation_set's binary_logloss: 0.0636557[2]	validation_set's binary_logloss: 0.0637513[1]	validation_set's binary_logloss: 0.0646903[1]	validation_set's binary_logloss: 0.0655738[1]	validation_set's binary_logloss: 0.0647049[1]	validation_set's binary_logloss: 0.0650748[1]	validation_set's binary_logloss: 0.0656645[1]	validation_set's binary_logloss: 0.0632047[1]	validation_set's binary_logloss: 0.0679754[1]	validation_set's binary_logloss: 0.0699245[1]	validation_set's binary_logloss: 0.0685548









Training until validation scores don't improve for 4 rounds
[1]	validation_set's binary_logloss: 0.0772675
[1]	validation_set's binary_logloss: 0.0665752Training until validation scores don't improve for 4 rounds[1]	validation_set's binary_logloss: 0.064328Training until validation scores don't improve for 4 roundsTraining until 

Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']


[16]	validation_set's binary_logloss: 0.0636234[15]	validation_set's binary_logloss: 0.0676264[15]	validation_set's binary_logloss: 0.0670209[15]	validation_set's binary_logloss: 0.0535864[15]	validation_set's binary_logloss: 0.0655554

categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']


...

[2]	validation_set's binary_logloss: 0.0625557[9]	validation_set's binary_logloss: 0.0643128[6]	validation_set's binary_logloss: 0.0639811


[7]	validation_set's binary_logloss: 0.0669135[3]	validation_set's binary_logloss: 0.0699886[11]	validation_set's binary_logloss: 0.0626342[12]	validation_set's binary_logloss: 0.0656897[10]	validation_set's binary_logloss: 0.0669767[12]	validation_set's binary_logloss: 0.067185[10]	validation_set's binary_logloss: 0.0639157[6]	validation_set's binary_logloss: 0.0680388







[3]	validation_set's binary_logloss: 0.0620061[10]	validation_set's binary_logloss: 0.0643619
[7]	validation_set's binary_logloss: 0.0640302

[12]	validation_set's binary_logloss: 0.0627126[4]	validation_set's binary_logloss: 0.0692421[8]	validation_set's binary_logloss: 0.066685[11]	validation_set's binary_logloss: 0.0668304
[13]	validation_set's binary_logloss: 0.0657493[7]	validation_set's binary_logloss: 0.067294[11]	validation_set's binary_logloss: 0.0640371





[4]	va

Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']


[13]	validation_set's binary_logloss: 0.0663364[15]	validation_set's binary_logloss: 0.0658986[10]	validation_set's binary_logloss: 0.0663112[15]	validation_set's binary_logloss: 0.0669688[6]	validation_set's binary_logloss: 0.0684924[9]	validation_set's binary_logloss: 0.0660304[6]	validation_set's binary_logloss: 0.0616009






[7]	validation_set's binary_logloss: 0.0616384[7]	validation_set's binary_logloss: 0.0680126[14]	validation_set's binary_logloss: 0.0662754
[11]	validation_set's binary_logloss: 0.0663102
[10]	validation_set's binary_logloss: 0.0656126[16]	validation_set's binary_logloss: 0.0668784

[16]	validation_set's binary_logloss: 0.0657052


[8]	validation_set's binary_logloss: 0.0615761[8]	validation_set's binary_logloss: 0.067722[12]	validation_set's binary_logloss: 0.0667129[11]	validation_set's binary_logloss: 0.065317[15]	validation_set's binary_logloss: 0.0661989Early stopping, best iteration is:
[12]	validation_set's binary_logloss: 0.0656897
[17]	validation_set

Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']


[19]	validation_set's binary_logloss: 0.0657369[21]	validation_set's binary_logloss: 0.0667889[1]	validation_set's binary_logloss: 0.0680586[13]	validation_set's binary_logloss: 0.0667814
[15]	validation_set's binary_logloss: 0.0642342[1]	validation_set's binary_logloss: 0.064714




Training until validation scores don't improve for 4 roundsTraining until validation scores don't improve for 4 rounds

[20]	validation_set's binary_logloss: 0.0656506[22]	validation_set's binary_logloss: 0.066783[16]	validation_set's binary_logloss: 0.0639979[14]	validation_set's binary_logloss: 0.0667869



[2]	validation_set's binary_logloss: 0.0677185[2]	validation_set's binary_logloss: 0.0641161

[21]	validation_set's binary_logloss: 0.0654102
[17]	validation_set's binary_logloss: 0.0636758[23]	validation_set's binary_logloss: 0.066776[15]	validation_set's binary_logloss: 0.0666585


[3]	validation_set's binary_logloss: 0.0638473[3]	validation_set's binary_logloss: 0.0674707

[22]	validation_set's bin

We want to evaluate our model on the set of leaders in July 2017, 36 months prior to the most recent month in our dataset. Doing so is as simple as `modeler.evaluate()`. For each time horizon ("Lead Length") we obtain a variety of binary classification metrics, including the [area under the receiver operating characteristic curve (AUROC)](https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve). Our AUROC values hover around the low-to-mid 0.80's over the first 9 months into the future and around the low-to-mid 0.70's for longer time horizons. The [concordance index ("C-Index")](https://statisticaloddsandends.wordpress.com/2019/10/26/what-is-harrells-c-index/) provides a metric for model performance over all time horizons. Like AUROC, a C-index of 0.5 indicates a useless model, while a C-index of 1.0 is perfect.

In [10]:
metrics = modeler.evaluate()
metrics

,AUROC,Predicted Share,Actual Share,True Positives,False Negatives,False Positives,True Negatives,Other Metrics:,C-Index
Lead Length,,,,,,,,,
1,0.844560,0.990449,0.984694,193,0,3,0,,0.7009708737864078
2,0.924007,0.980002,0.984694,192,1,3,0,,
3,0.790576,0.969467,0.974490,191,0,4,1,,
4,0.837491,0.949735,0.964286,188,1,5,2,,
5,0.788496,0.933758,0.938776,183,1,10,2,,
6,0.821036,0.917104,0.928571,181,1,10,4,,
7,0.834254,0.899760,0.923469,179,2,10,5,,
8,0.832402,0.882514,0.913265,176,3,11,6,,
9,0.802177,0.866117,0.892857,171,4,14,7,,


The model we trained used default values for hyperparameters such as the maximum number of leaves per tree and the minimum number of observations in each leaf. There are likely other hyperparameter values that would yield better performance for each future period. Searching for those values and returning the best candidate set for each future period (according to performance on a validation set) is as simple as `modeler.hyperoptimize()`. The number of sets to consider, `n_trials`, defaults to 64, but we'll specify a value of 4 to accelerate this example.

In [11]:
N_TRIALS = 4
params = modeler.hyperoptimize(n_trials=N_TRIALS)
params

[I 2020-08-28 16:30:20,509] Trial 0 finished with value: 0.06059920447311717 and parameters: {'num_iterations': 94, 'learning_rate': 0.4032261012568846, 'num_leaves': 121, 'max_depth': 19, 'min_data_in_leaf': 369, 'min_sum_hessian_in_leaf': 0.1364906231563803, 'bagging_freq': 0, 'bagging_fraction': 0.7271848812213663, 'feature_fraction': 0.6779042758880096, 'lambda_l1': 55.32233457830071, 'lambda_l2': 42.84656691186624, 'min_gain_to_split': 0.07127895884983065, 'min_data_per_group': 431, 'max_cat_threshold': 70, 'cat_l2': 41.37300385050969, 'cat_smooth': 1229.1342054498475, 'max_cat_to_onehot': 16, 'max_bin': 326, 'min_data_in_bin': 37}. Best is trial 0 with value: 0.06059920447311717.
[I 2020-08-28 16:30:20,751] Trial 1 finished with value: 0.06054656727625292 and parameters: {'num_iterations': 67, 'learning_rate': 0.03746294609752311, 'num_leaves': 158, 'max_depth': 12, 'min_data_in_leaf': 32, 'min_sum_hessian_in_leaf': 0.14511985173887948, 'bagging_freq': 0, 'bagging_fraction': 0.67

{0: {'num_iterations': 91,
  'learning_rate': 0.08081207670528569,
  'num_leaves': 45,
  'max_depth': 30,
  'min_data_in_leaf': 52,
  'min_sum_hessian_in_leaf': 0.14564408551350866,
  'bagging_freq': 1,
  'bagging_fraction': 0.7421054227078454,
  'feature_fraction': 0.9553626362706973,
  'lambda_l1': 33.48323382484937,
  'lambda_l2': 25.91037545717554,
  'min_gain_to_split': 0.039619274608901967,
  'min_data_per_group': 284,
  'max_cat_threshold': 377,
  'cat_l2': 11.820566283956346,
  'cat_smooth': 119.75451760945361,
  'max_cat_to_onehot': 11,
  'max_bin': 150,
  'min_data_in_bin': 10,
  'objective': 'binary'},
 1: {'num_iterations': 98,
  'learning_rate': 0.48221228650561615,
  'num_leaves': 222,
  'max_depth': 14,
  'min_data_in_leaf': 343,
  'min_sum_hessian_in_leaf': 0.16336286068921954,
  'bagging_freq': 0,
  'bagging_fraction': 0.9794991100182464,
  'feature_fraction': 0.63728942034678,
  'lambda_l1': 38.02920861460169,
  'lambda_l2': 39.83162229799324,
  'min_gain_to_split': 0

Using our new hyperparameter sets is as simple as passing them to build_model. Because the number of trees to train was one of our hyperparameters, we don't use early stopping and don't need a validation set.

In [12]:
modeler.build_model(n_intervals=TEST_INTERVALS, params=params)

Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']
Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'n

We can now evaluate our new model and directly compare it to our default model. The hyperparameters that performed best on a validation set don't always outperform the default parameters on the test set, but in this case they do. We see that our new hyperparameters yield higher AUROCs and a higher C-Index than the defaults.

In [13]:
hyperoptimized_metrics = modeler.evaluate()
hyperoptimized_metrics

,AUROC,Predicted Share,Actual Share,True Positives,False Negatives,False Positives,True Negatives,Other Metrics:,C-Index
Lead Length,,,,,,,,,
1,0.913644,0.985811,0.984694,193,0,3,0,,0.7276699029126213
2,0.892919,0.976030,0.984694,192,1,3,0,,
3,0.805236,0.965558,0.974490,191,0,4,1,,
4,0.857143,0.945799,0.964286,189,0,5,2,,
5,0.776268,0.923812,0.938776,183,1,8,4,,
6,0.821821,0.903341,0.928571,181,1,9,5,,
7,0.843831,0.885503,0.923469,179,2,7,8,,
8,0.831745,0.872482,0.913265,176,3,9,8,,
9,0.844354,0.857054,0.892857,170,5,11,10,,


To use our new hyperparameters to get forecasts, we re-train our model with the most recent 36 months included. However, because we only obtained hyperparameters for time horizons up through 36 months, we can only forecast that far ahead.

In [14]:
data_processor.config["TEST_INTERVALS"] = 0
data_processor.build_reserved_cols()
modeler = LGBSurvivalModeler(data=data_processor.data)
modeler.build_model(n_intervals=TEST_INTERVALS, params=params)

Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'nochange_recent', 'prev_conflict', 'pt_attempt', 'pt_suc', 'ref_ant', 'ref_recent', 'tenure_months', 'victory_recent', 'year']
Found `num_iterations` in params. Will use it instead of argument
categorical_feature in Dataset is overridden.
New categorical_feature is ['age', 'anticipation', 'change_recent', 'country', 'defeat_recent', 'delayed', 'direct_recent', 'elected', 'election_now', 'election_recent', 'exec_ant', 'exec_recent', 'government', 'indirect_recent', 'irreg_lead_ant', 'lead_recent', 'leg_ant', 'leg_recent', 'male', 'militarycareer', 'month', 'n

Obtaining forecasts is as easy as before.

In [15]:
forecasts = modeler.forecast()
forecasts.columns = custom_forecast_headers[:len(forecasts.columns)]
forecasts

,Aug 2020,Sep 2020,Oct 2020,Nov 2020,Dec 2020,Jan 2021,Feb 2021,Mar 2021,Apr 2021,May 2021,...,Oct 2022,Nov 2022,Dec 2022,Jan 2023,Feb 2023,Mar 2023,Apr 2023,May 2023,Jun 2023,Jul 2023
country-leader,,,,,,,,,,,,,,,,,,,,,
Afghanistan: Ashraf Ghani,0.992963,0.981697,0.961879,0.953849,0.946131,0.935087,0.925610,0.905549,0.899499,0.893231,...,0.743198,0.733864,0.724791,0.715469,0.713219,0.704937,0.697270,0.688140,0.678353,0.668372
Albania: Rama,0.994842,0.988446,0.979828,0.977302,0.963571,0.935421,0.931109,0.926147,0.922034,0.900521,...,0.564641,0.553395,0.544010,0.537616,0.532650,0.520952,0.517667,0.512088,0.506248,0.501393
Algeria: Tebboune,0.987983,0.982174,0.972357,0.956653,0.951243,0.943842,0.936244,0.932785,0.928903,0.925106,...,0.768865,0.759895,0.750201,0.741392,0.735561,0.729317,0.726451,0.718173,0.711028,0.706557
Andorra: Espot Zamora,0.991076,0.984545,0.976307,0.973311,0.967607,0.957102,0.952409,0.948324,0.934205,0.926678,...,0.691257,0.680294,0.669543,0.659521,0.652747,0.647812,0.644114,0.631460,0.530091,0.519022
Angola: Lourenco,0.996215,0.995258,0.989787,0.987044,0.984465,0.981338,0.979790,0.977643,0.976260,0.974738,...,0.858105,0.848666,0.841719,0.834672,0.830514,0.825537,0.821610,0.813451,0.806598,0.802034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela: Nicolas Maduro,0.995536,0.993568,0.991869,0.990174,0.986145,0.984045,0.983489,0.980619,0.970149,0.968902,...,0.825330,0.814304,0.801257,0.788188,0.771623,0.749391,0.479278,0.472753,0.466025,0.452853
Vietnam: Phu Trong,0.989908,0.984599,0.974808,0.967434,0.964458,0.950400,0.870821,0.854662,0.840046,0.836471,...,0.600065,0.592665,0.584874,0.577657,0.557608,0.551623,0.548083,0.541254,0.534234,0.526488
Yemen: Houthi,0.992958,0.990865,0.982822,0.970978,0.967616,0.956121,0.946688,0.894732,0.891542,0.888650,...,0.744439,0.734522,0.723578,0.712411,0.698817,0.686234,0.678565,0.670468,0.660760,0.656959
